In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float')/255.0
x_test = x_test.astype('float')/255.0

x_train = np.reshape(x_train, (60000, 784))
x_test = np.reshape(x_test, (10000, 784))


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train_noisy = x_train + np.random.rand(60000, 784) * 0.9
x_test_noisy = x_test + np.random.rand(10000, 784) * 0.9
x_train_noisy = np.clip(x_train_noisy, 0. , 1.)
x_test_noisy = np.clip(x_test_noisy, 0. , 1.)

In [0]:
def plot(x, p, labels = False):
    plt.figure(figsize = (30  , 10))
    for i in range(1, 10):
        plt.subplot(1, 10, i+1)
        plt.imshow(x[i+20].reshape(28,28), cmap = 'binary')
        plt.xticks([])
        plt.yticks([])
        if labels:
          plt.xlabel(np.argmax(p[i]))
    plt.show()

In [0]:
plot(x_train, None)

In [0]:
plot(x_train_noisy, None)

In [0]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if (logs.get('accuracy') > 0.97):
            print("\n Achieved more than 97% accuracy, Hence terminating training!!!")
            self.model.stop_training = True
callbacks = myCallback()

In [0]:
classifier = Sequential([
    Dense(256, activation = 'relu', input_shape = (784,)),
    Dense(256, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

classifier.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
classifier.summary()

In [0]:
classifier.fit(x_train, y_train, epochs = 20, callbacks = [callbacks])

In [0]:
loss, acc = classifier.evaluate(x_train, y_train)
print('Training accuracy:{}\n'.format(acc))
loss, acc = classifier.evaluate(x_test, y_test)
print('Test accuracy:{}'.format(acc))

In [0]:
print('Evaluating classifier on noisy images')
loss, acc = classifier.evaluate(x_train_noisy, y_train)
print('\nTraining accuracy:{}'.format(acc))
loss, acc = classifier.evaluate(x_test_noisy, y_test)
print('\nTest accuracy:{}'.format(acc))


This is the implementation of deep autoencoder consisting of six **hidden** layers: three encoding layers and three decoding layers. More layers can be added but for this paritcular dataset, current config is sufficient.

In [0]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [0]:
autoencoder.fit(x_train_noisy, 
                x_train, 
                epochs = 100, 
                batch_size = 256, 
                verbose = True, 
                validation_data = (x_test_noisy, x_test),
                callbacks = [
                             EarlyStopping(monitor = 'val_loss', patience = 5),
                             LambdaCallback(on_epoch_end = lambda epoch, logs: print('Epoch {}: {}'.format(epoch, logs['val_loss'])))
                             ]
                )

In [0]:
pred = autoencoder.predict(x_test_noisy)

In [0]:
plot(x_test_noisy, None)

In [0]:
plot(pred, None)

Combined classifier and autoencoder model:

In [0]:
input_img = Input(shape = (784,))
x = autoencoderv2(input_img)
y = classifier(x)

denoise_and_classifyv2 = Model(input_img, y)

In [0]:
predictions = denoise_and_classify(x_test_noisy)
plot(x_test_noisy, predictions, True)

In [0]:
plot(x_test, to_categorical(y_test), True)

In [0]:
temp2 = autoencoderv2.predict(x_test_noisy)
acc = classifier.evaluate(temp2)
print(acc)

In [0]:
plot(predv2, None)